### First step: manipulate de DB

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
sqlitedb_path = '/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw/raw_data_project_m1.db'

In [3]:
engine = create_engine(f'sqlite:///{sqlitedb_path}')


In [4]:
# list all tables in database
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", engine)

,name
0,personal_info
1,country_info
2,career_info
3,poll_info


In [5]:
# query one of 0,1,2 & 3
df_personal_info = pd.read_sql_query("select * from personal_info", engine)
country_info = pd.read_sql_query("select * from country_info", engine)
career_info = pd.read_sql_query("select * from career_info", engine)
poll_info = pd.read_sql_query("select * from poll_info", engine)



In [6]:
print(len(df_personal_info))
print(len(country_info))
print(len(career_info))
print(len(poll_info))


9649
9649
9649
9649


In [7]:
#Letś do a merge to have a unique dataset

df_personal_info
country_info
career_info
poll_info

a = df_personal_info.merge(country_info, left_on='uuid', right_on='uuid')
b = a.merge(career_info, left_on='uuid', right_on='uuid')
final_sql= b.merge(poll_info, left_on='uuid', right_on='uuid')


In [8]:
final_sql.head()

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65,AT,countryside,no,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39,AT,city,None,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65,AT,Country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...


In [9]:
#letś save the file from the sql

import os

mainpath = "/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw"
filename= "download_sql"
fullpath= os.path.join(mainpath,filename)
final_sql.to_csv(fullpath+ ".csv", index=False)

### Second step : extraction from the API

In [10]:
import requests
import pandas as pd

final_sql = pd.read_csv("/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw/download_sql.csv")


response = requests.get('http://api.dataatwork.org/v1/jobs?limit=600') #saco el máximo de jobs
json_data = response.json()
#letś save the file from the sql

import os

mainpath = "/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw"
filename= "download_sql"
fullpath= os.path.join(mainpath,filename)
final_sql.to_csv(fullpath+ ".csv", index=False)
json_data[0:3]

len(json_data) #tenemos un maximo de 500 que es lo que nos permite


501

In [11]:
# we need to extract the jobs description from another API, that api requires an input

norm_job_codes = final_sql['normalized_job_code'].unique().tolist()
norm_job_dic = {}

In [12]:
norm_job_dic

{}

In [13]:
#  we create a loop to iterate all the dic values

for i in norm_job_codes[1:]: #to prevent errors , let's skip none and then we will add it. You can write a  if error pass.
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    norm_job_dic[i] = (response.json())['normalized_job_title']
norm_job_dic['none'] = 'none' # adding the none

In [14]:
pd.DataFrame(norm_job_dic.items()) # extract the dic values to create a dataframe
job_codes = pd.DataFrame(norm_job_dic.items(), columns=['normalized_job_code', 'normalized_job_title'])
job_codes.head()

,normalized_job_code,normalized_job_title
0,861a9b9151e11362eb3c77ca914172d0,automatic data processing planner
1,049a3f3a2b5f85cb2971ba77ad66e10c,data coordinator
2,f4b2fb1aa40f661488e2782b6d57ad2f,database developer
3,27af8700f5577cec835acee2cb90a2ff,data entry specialist
4,c1b670eba9ccb65e7c99f7da116d5b9c,database architect


In [15]:
job_codes.describe()

,normalized_job_code,normalized_job_title
count,157,157
unique,157,157
top,970eb38c7b3eb080466052ea994af189,database specialist
freq,1,1


In [16]:
job_codes.columns.values.tolist() # check if the columns were correctly saved

['normalized_job_code', 'normalized_job_title']

In [17]:
# Let's add that new info to our dataframe and save it as csv

final_dataset = final_sql.merge(job_codes, left_on='normalized_job_code', right_on='normalized_job_code')

In [18]:
import os
mainpath = "/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw"
filename= "final_dataset"
fullpath= os.path.join(mainpath,filename)
final_dataset.to_csv(fullpath+ ".csv", index=False)

### Scrapping Eurostat

In [19]:
final_dataset = pd.read_csv("/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw/final_dataset.csv")

In [20]:
final_dataset.head() # the key is country_code

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,normalized_job_title
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,48 years old,FeMale,NO,40_65,AT,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,automatic data processing planner
2,9ffc7390-dba5-0133-259b-0a81e8b09a82,43 years old,male,YES,40_65,BE,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,It increases dependence on the state,automatic data processing planner
3,a6361230-da58-0133-cd5a-0a81e8b09a82,43 years old,Male,yES,40_65,BE,urban,medium,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote against it,A basic income would not affect my work choices,None of the above,Only the people who need it most should get so...,automatic data processing planner
4,1e2b4750-d987-0133-3c7c-0a81e8b09a82,52 years old,female,YES,40_65,BG,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I have heard just a little about it,I would probably vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs,It might encourage people to stop working,automatic data processing planner


In [202]:
import requests

url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
html[:500]

b'\xef\xbb\xbf<!DOCTYPE html>\n<html lang="en" dir="ltr" class="client-nojs">\n<head>\n<meta http-equiv="X-UA-Compatible" content="Ie=Edge,chrome=1"/> <meta charset="UTF-8" /><title>Glossary:Country codes - Statistics Explained</title>\n<meta name="generator" content="MediaWiki 1.22.5" />\n<link rel="shortcut icon" href="https://ec.europa.eu/eurostat/estat-theme/images/favicon.ico" />\n<link rel="search" type="application/opensearchdescription+xml" href="/eurostat/statistics-explained/opensearch_desc.php" title='

In [203]:
from bs4 import BeautifulSoup

# lxml is the parsing module
soup = BeautifulSoup(html, 'lxml')
table = soup.find_all('td')



In [204]:
table = list(table)

import re

CLEAN = re.compile('\s+()')

table = [CLEAN.sub(' ', a.text).strip() for a in table]

In [205]:
import re

CLEAN = re.compile('[\s+()]')

table = [CLEAN.sub(' ', a.text).strip() for a in table]



In [206]:
len(table)

175

In [207]:
table= list(table)

In [208]:
for i in table: #iterable to eliminate blanks
    if i == "": 
        table.remove(i)

In [209]:
for i in table:
    if i == "": 
        table.remove(i)

In [210]:
len(table)

144

In [211]:
type(table)

list

In [212]:
table.append("Great Britain")
table.append("GB")
table.append("Greece")
table.append("GR")


In [213]:
def particionar_lista(table,n):
    return([table[i*n:i*n+n] for i in range (len(table))])

In [214]:
table_function = particionar_lista(table,2)

In [215]:
print(table_function)
   

[['Belgium', 'BE'], ['Greece', 'EL'], ['Lithuania', 'LT'], ['Portugal', 'PT'], ['Bulgaria', 'BG'], ['Spain', 'ES'], ['Luxembourg', 'LU'], ['Romania', 'RO'], ['Czechia', 'CZ'], ['France', 'FR'], ['Hungary', 'HU'], ['Slovenia', 'SI'], ['Denmark', 'DK'], ['Croatia', 'HR'], ['Malta', 'MT'], ['Slovakia', 'SK'], ['Germany', 'DE'], ['Italy', 'IT'], ['Netherlands', 'NL'], ['Finland', 'FI'], ['Estonia', 'EE'], ['Cyprus', 'CY'], ['Austria', 'AT'], ['Sweden', 'SE'], ['Ireland', 'IE'], ['Latvia', 'LV'], ['Poland', 'PL'], ['United Kingdom', 'UK'], ['Iceland', 'IS'], ['Norway', 'NO'], ['Liechtenstein', 'LI'], ['Switzerland', 'CH'], ['Montenegro', 'ME'], ['North Macedonia', 'MK'], ['Albania', 'AL'], ['Serbia', 'RS'], ['Turkey', 'TR'], ['Bosnia and Herzegovina', 'BA'], ['Kosovo*[1]', 'XK[2]'], ['Armenia', 'AM'], ['Belarus', 'BY'], ['Georgia', 'GE'], ['Azerbaijan', 'AZ'], ['Moldova', 'MD'], ['Ukraine', 'UA'], ['Algeria', 'DZ'], ['Lebanon', 'LB'], ['Syria', 'SY'], ['Egypt', 'EG'], ['Libya', 'LY'], ['Tun

In [216]:
type(table_function)

list

In [217]:
df = pd.DataFrame(table_function , columns=["country","country_code"])

In [218]:
df.head()

,country,country_code
0,Belgium,BE
1,Greece,EL
2,Lithuania,LT
3,Portugal,PT
4,Bulgaria,BG


In [219]:
df = df.dropna() #drop the Nan
    

In [220]:
df_scrap = df[["country_code","country"]] #reorder the dataframe for the join

In [221]:
df_scrap = df_scrap['country'].replace('Great Britain', 'United Kingdom')

In [222]:
final_dataset.head(3)

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,normalized_job_title
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,48 years old,FeMale,NO,40_65,AT,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,automatic data processing planner
2,9ffc7390-dba5-0133-259b-0a81e8b09a82,43 years old,male,YES,40_65,BE,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,It increases dependence on the state,automatic data processing planner


In [223]:
df_scrap = df[["country_code","country"]] 

In [224]:
final_dataset_raw = pd.merge( final_dataset , df_scrap , how = 'left' , on = 'country_code')

In [225]:
final_dataset_raw.head(3)



,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,normalized_job_title,country
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner,Austria
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,48 years old,FeMale,NO,40_65,AT,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,automatic data processing planner,Austria
2,9ffc7390-dba5-0133-259b-0a81e8b09a82,43 years old,male,YES,40_65,BE,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,It increases dependence on the state,automatic data processing planner,Belgium


In [226]:
import os
mainpath = "/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw"
filename= "final_dataset_raw"
fullpath= os.path.join(mainpath,filename)
final_dataset_raw.to_csv(fullpath+ ".csv", index=False)

### Data Wrangling

In [227]:
final_dataset_raw.head()

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,normalized_job_title,country
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner,Austria
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,48 years old,FeMale,NO,40_65,AT,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,automatic data processing planner,Austria
2,9ffc7390-dba5-0133-259b-0a81e8b09a82,43 years old,male,YES,40_65,BE,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,It increases dependence on the state,automatic data processing planner,Belgium
3,a6361230-da58-0133-cd5a-0a81e8b09a82,43 years old,Male,yES,40_65,BE,urban,medium,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote against it,A basic income would not affect my work choices,None of the above,Only the people who need it most should get so...,automatic data processing planner,Belgium
4,1e2b4750-d987-0133-3c7c-0a81e8b09a82,52 years old,female,YES,40_65,BG,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I have heard just a little about it,I would probably vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs,It might encourage people to stop working,automatic data processing planner,Bulgaria


In [228]:
final_dataset_raw.rural.unique() #let ś understand the data inside the rural column

array(['urban', 'Country', 'Non-Rural', 'city', 'rural', 'countryside'],
      dtype=object)

In [229]:
final_dataset_raw = final_dataset_raw.replace({"urban" :"City", "Country":"Rural","Non-Rural":"City","rural":"Rural","countryside":"Rural" , "city":"City" }) # homogenise the data into two main categories

In [230]:
final_dataset_raw.rural.unique() #check

array(['City', 'Rural'], dtype=object)

In [231]:
final_dataset_def  =final_dataset_raw[["country", "normalized_job_title", "rural", "uuid"]]

In [232]:
import os
mainpath = "/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw"
filename= "final_dataset_def"
fullpath= os.path.join(mainpath,filename)
final_dataset_def.to_csv(fullpath+ ".csv", index=False)

In [233]:
final_dataset_def =  final_dataset_def.rename(columns = {"country" : "Country" , "normalized_job_title" : "Job Title" , "rural" :"Rural"})

final_dataset_def[490:500]

,Country,Job Title,Rural,uuid
490,Slovakia,data security analyst,Rural,8a157220-da38-0133-48e8-0a81e8b09a82
491,Slovakia,data security analyst,Rural,b3375cf0-d9b3-0133-2d96-0a81e8b09a82
492,Austria,database marketing analyst,Rural,a96ed910-de55-0133-2582-0a81e8b09a82
493,Belgium,database marketing analyst,Rural,d73155f0-d9ba-0133-fe9f-0a81e8b09a82
494,Belgium,database marketing analyst,City,c75ffdb0-d95d-0133-765b-0a81e8b09a82
495,Czechia,database marketing analyst,City,7e5f95e0-d9b4-0133-d2e2-0a81e8b09a82
496,Germany,database marketing analyst,City,d129d530-db58-0133-fe1b-0a81e8b09a82
497,Germany,database marketing analyst,Rural,a6086e10-da2d-0133-81c1-0a81e8b09a82
498,Germany,database marketing analyst,City,162fbec0-dce4-0133-d5c7-0a81e8b09a82
499,Germany,database marketing analyst,Rural,523511b0-dd70-0133-03fc-0a81e8b09a82


In [234]:
df = final_dataset_def.groupby(["Country", "Job Title", "Rural"]).size().reset_index(name="Quantity")

In [235]:
df.head(10)

,Country,Job Title,Rural,Quantity
0,Austria,automatic data processing customer liaison adp...,City,1
1,Austria,automatic data processing customer liaison adp...,Rural,1
2,Austria,automatic data processing planner,City,1
3,Austria,automatic data processing planner,Rural,1
4,Austria,automatic data processing systems security spe...,City,1
5,Austria,automatic data processing systems security spe...,Rural,1
6,Austria,clinical data specialist,Rural,1
7,Austria,computer or data processing systems consultant,City,2
8,Austria,computer or data processing systems consultant,Rural,1
9,Austria,crime data specialist,City,1


In [257]:


df["Percentage"] = df["Quantity"] / df["Quantity"].sum()
df["Percentage"]  = df["Percentage"].round(8)

df["Percentage"].max()

0.0022799

In [258]:
df.tail(50)

,Country,Job Title,Rural,Quantity,Percentage
2358,Sweden,data officer,City,2,0.000351
2359,Sweden,data processing manager,City,2,0.000351
2360,Sweden,data processing manager,Rural,1,0.000175
2361,Sweden,data reduction technician,City,1,0.000175
2362,Sweden,data scientist,City,2,0.000351
2363,Sweden,data security administrator,City,1,0.000175
2364,Sweden,data security analyst,City,1,0.000175
2365,Sweden,data systems manager,City,1,0.000175
2366,Sweden,data transcriber,Rural,1,0.000175
2367,Sweden,data typist,City,2,0.000351


In [250]:
most_demanded_jobs = df.sort_values(by = "Percentage" , ascending= False )

In [251]:
most_demanded_jobs.head(10)

,Country,Job Title,Rural,Quantity,Percentage
749,Germany,data officer,City,13,0.0023
1058,Great Britain,database tester,City,12,0.0021
1395,Italy,data modeling specialist,City,12,0.0021
1392,Italy,data miner,City,12,0.0021
816,Germany,database consultant,City,12,0.0021
542,France,data modeling specialist,City,12,0.0021
1849,Poland,geographic information systems data specialist...,City,11,0.0019
829,Germany,database management system specialist dbms spe...,City,11,0.0019
1775,Poland,data storage specialist,City,11,0.0019
1418,Italy,data scientist,City,11,0.0019


In [252]:
df.tail(50)

,Country,Job Title,Rural,Quantity,Percentage
2358,Sweden,data officer,City,2,0.0004
2359,Sweden,data processing manager,City,2,0.0004
2360,Sweden,data processing manager,Rural,1,0.0002
2361,Sweden,data reduction technician,City,1,0.0002
2362,Sweden,data scientist,City,2,0.0004
2363,Sweden,data security administrator,City,1,0.0002
2364,Sweden,data security analyst,City,1,0.0002
2365,Sweden,data systems manager,City,1,0.0002
2366,Sweden,data transcriber,Rural,1,0.0002
2367,Sweden,data typist,City,2,0.0004


In [253]:
type(df['Percentage'])

pandas.core.series.Series

In [254]:
type(df['Percentage'])

pandas.core.series.Series

In [255]:
df['Percentage'] = df['Percentage'].astype(float).map("{:.2%}".format)


    


In [256]:
df.tail(20)

,Country,Job Title,Rural,Quantity,Percentage
2388,Sweden,database marketing analyst,City,2,0.04%
2389,Sweden,database modeler,City,2,0.04%
2390,Sweden,database programmer,City,1,0.02%
2391,Sweden,database security administrator,City,2,0.04%
2392,Sweden,database security administrator,Rural,1,0.02%
2393,Sweden,database software technician,City,1,0.02%
2394,Sweden,database specialist,City,2,0.04%
2395,Sweden,database tester,City,1,0.02%
2396,Sweden,databases computer consultant,City,1,0.02%
2397,Sweden,engineer system electronic data processing eng...,Rural,1,0.02%


In [245]:
mainpath = "/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw"
filename= "most_demanded_jobs"
fullpath= os.path.join(mainpath,filename)
most_demanded_jobs.to_csv(fullpath+ ".csv", index=False)

In [89]:
mainpath = "/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw"
filename= "df_for_visualisation"
fullpath= os.path.join(mainpath,filename)
df.to_csv(fullpath+ ".csv", index=False)

In [90]:
test = pd.read_csv("/home/jc/Escritorio/Proyectos_finales_ironhack/ih_datamadpt0420_project_m1/data/raw/df_for_visualisation.csv")

In [91]:
test.tail()

,Country,Job Title,Rural,Quantity,Percentage
2403,Sweden,head of information systems and data processing,City,1,0.041528
2404,Sweden,naval tactical data systemcombat information c...,City,1,0.041528
2405,Sweden,oracle database administrator oracle dba,City,1,0.041528
2406,Sweden,sql database administrator,Rural,1,0.041528
2407,Sweden,survey data technician,City,1,0.041528


In [92]:
filtered_country = test[test['Country']=='Greece']

In [93]:
filtered_country.head()

,Country,Job Title,Rural,Quantity,Percentage
1108,Greece,automatic data processing customer liaison adp...,City,2,0.083056
1109,Greece,automatic data processing customer liaison adp...,Rural,1,0.041528
1110,Greece,automatic data processing planner,City,1,0.041528
1111,Greece,automatic data processing planner adp planner,City,3,0.124585
1112,Greece,automatic data processing systems security spe...,City,3,0.124585


In [260]:
test['Country'].unique().tolist()

['Austria',
 'Belgium',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Great Britain',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden']